# 📊 Trader Behavior vs Bitcoin Market Sentiment

Full analysis including visual insights, top trader behaviors, leverage, and predictive modeling.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

sns.set(style="whitegrid")

# Load data
historical_df = pd.read_csv("historical_data.csv")
fear_greed_df = pd.read_csv("fear_greed_index.csv")

# Preprocess and merge
historical_df['date'] = pd.to_datetime(historical_df['Timestamp IST'], format='%d-%m-%Y %H:%M').dt.date
fear_greed_df['date'] = pd.to_datetime(fear_greed_df['date']).dt.date
merged_df = pd.merge(historical_df, fear_greed_df[['date', 'classification']], on='date', how='left')


## 🔍 Summary Statistics

In [ ]:
merged_df.describe(include='all')

## 📈 Average PnL by Sentiment

In [ ]:
sns.barplot(
    data=merged_df.groupby('classification')['Closed PnL'].mean().reset_index(),
    x='classification', y='Closed PnL', palette='viridis')
plt.title("Avg. Closed PnL by Market Sentiment")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 📦 PnL Distribution by Sentiment

In [ ]:
sns.boxplot(data=merged_df, x='classification', y='Closed PnL', showfliers=False, palette='Set2')
plt.title("PnL Distribution by Sentiment")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 💼 Top Traders by Sentiment

In [ ]:
top_traders = merged_df.groupby("Account")["Closed PnL"].sum().sort_values(ascending=False).head(10).index
top_trader_df = merged_df[merged_df["Account"].isin(top_traders)]
sns.boxplot(data=top_trader_df, x="classification", y="Closed PnL", hue="Account", showfliers=False)
plt.title("Top 10 Traders' PnL by Sentiment")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 🔮 Predictive Model – Will a Trade Be Profitable?

In [ ]:

merged_df['profitable'] = (merged_df['Closed PnL'] > 0).astype(int)
categorical_cols = ['Side', 'Direction', 'classification', 'Coin']
le = LabelEncoder()
for col in categorical_cols:
    merged_df[col] = le.fit_transform(merged_df[col])

features = ['Execution Price', 'Size USD', 'Start Position', 'Fee', 'Side', 'Direction', 'classification']
X = merged_df[features].replace([np.inf, -np.inf], 0).fillna(0)
y = merged_df['profitable']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))


In [ ]:
pd.Series(model.coef_[0], index=X.columns).sort_values(ascending=False)